<a href="https://colab.research.google.com/github/snaiws/NLP_project/blob/RoBERTa%2Fbaseline/Model_roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# transformer, datset 설치
!pip install transformers datasets
# Hyperparameter 탐색: optuna 설치
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.8 MB/s 
     |████████████████████████████████| 346 kB 61.1 MB/s 
     |████████████████████████████████| 596 kB 39.2 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 42.1 MB/s 
     |████████████████████████████████| 212 kB 11.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.9 MB/s 
     |████████████████████████████████| 86 kB 8.1 MB/s 
     |████████████████████████████████| 140 kB 77.4 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
     |████████████████████████████████| 271 kB 70.5 MB/s 
     |████████████████████████████████| 144 kB 77.9 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

In [26]:
# tensorboard 패키지 설치
!pip install jupyter-tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for jupyter-tensorboard: filename=jupyter_tensorboard-0.2.0-py2.py3-none-any.whl size=15258 sha256=e9ba1f93af7363724632892fccb6d360e18115ac68855e172781c75cb1895007
  Stored in directory: /root/.cache/pip/wheels/59/14/ab/6d0bce449039ebdcbf45c1aff8c19153a68bf3f0492a32620e
Successfully built jupyter-tensorboard


In [27]:
# tensorboard extension 설치
%load_ext tensorboard

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import transformers
from transformers import EarlyStoppingCallback
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,  RandomSampler, SequentialSampler, random_split
from torch.nn.utils import clip_grad_norm_

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

from datasets import load_metric
from torch.utils.tensorboard import SummaryWriter
import os
import re
import pandas as pd
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt

In [4]:
#reset gpu cache
import gc
gc.collect()
torch.cuda.empty_cache()

In [5]:

# seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


In [10]:
# argument setting
train_batch_size = 58
eval_batch_size = 64
epochs=25
patience=2
model_checkpoint = "klue/roberta-base"

loss_fct = nn.CrossEntropyLoss()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
#tokenizer = BertTokenizer.from_pretrained("klue/roberta-small")
#tokenizer = BertTokenizer.from_pretrained("klue/roberta-base")
#tokenizer = BertTokenizer.from_pretrained("klue/roberta-large")
# customModel = Bert_seq(hidden_size=768, n_label=6)
# customOptimizer = AdamW(
#     customModel.parameters(), # update 대상 파라미터를 입력
#     lr=2.335281794951567e-05,
#     eps=1e-8
# )

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [8]:
cd /content/drive/MyDrive/AIBootCamp/NLP

/content/drive/MyDrive/AIBootCamp/NLP


In [9]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "data/klue-sts-v1.1")
print(f"My data directory : {_DATA_DIR}")

My current directory : /content/drive/MyDrive/AIBootCamp/NLP
My data directory : /content/drive/MyDrive/AIBootCamp/NLP/data/klue-sts-v1.1


In [12]:
# 파일 내 json 불러오기
df_train0 = pd.read_json('./data/klue-sts-v1.1/klue-sts-v1.1_train.json')
df_test0 = pd.read_json('./data/klue-sts-v1.1/klue-sts-v1.1_dev.json')

In [67]:
from sklearn.model_selection import train_test_split

class Preprocess:
    def __init__(self,train,test):
        self.train=train
        self.test=test

    def check_st(self, text):
      text = re.sub('a-zA-Z一-龥㐀-䶵豈-龎[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
      return text
    
    def robeta_base(self,dataset,mode=None):
      '''
      transform to the Input example
      '''
      #중복 제거
      dataset = dataset.drop_duplicates(['sentence1','sentence2']).reset_index(drop=True)

      sent_pairs = [sent1+' '+sent2 for sent1, sent2 in zip(dataset['sentence1'], dataset['sentence2'])]
      sent_pairs = list(map(self.check_st, sent_pairs))

      #real-label -> regression 활용 가능?
      r_score=[i['real-label'] for i in dataset['labels']]

      #custom label : int
      c_score=[round(i['real-label']) for i in dataset['labels']]

      #b_score label : [0,1] -> f1 scroe
      b_score = [i['binary-label'] for i in dataset['labels']]
      input_examples = pd.DataFrame({'guid': dataset['guid'], 'sent_pair' : sent_pairs, 'r_label' : r_score, 'c_label' : c_score, 'b_label' : b_score})

      if mode == 'train':
        train, valid = train_test_split(input_examples, test_size=0.1, random_state=seed)
        return train, valid

      return input_examples

In [70]:
pp=Preprocess(df_train0,df_test0)

train, valid = pp.robeta_base(df_train0, 'train')
test = pp.robeta_base(df_test0)

In [71]:
print(f"Train Dataset len: {len(train)}")
print(f"Valid Dataset len: {len(valid)}")
print(f"Test Dataset len: {len(test)}")

Train Dataset len: 10494
Valid Dataset len: 1167
Test Dataset len: 519


In [72]:
# Custom Dataset
class CustomDataset(Dataset):
    """
    - input_data: list of string
    - target_data: list of int
    """
    
    def __init__(self, input_data:list, target_data:list) -> None:
        self.X = input_data
        self.Y = target_data
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [73]:
train_dataset = CustomDataset(train.sent_pair.to_list(), train.c_label.to_list())
valid_dataset = CustomDataset(valid.sent_pair.to_list(), valid.c_label.to_list())
test_dataset = CustomDataset(test.sent_pair.to_list(), test.c_label.to_list())

In [83]:
# dataset = load_dataset('klue', task)
klue_sts_train = load_dataset("klue", "sts", split='train[:90%]')
klue_sts_valid = load_dataset("klue", "sts", split='train[-10%:]') # train의 10%를 validation set으로 사용
klue_sts_test = load_dataset("klue", "sts", split='validation')

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)
Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)
Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


In [84]:
# real-label을 label로 변경 
klue_sts_train = klue_sts_train.flatten()
klue_sts_train = klue_sts_train.rename_column('labels.real-label','label')
klue_sts_valid = klue_sts_valid.flatten()
klue_sts_valid = klue_sts_valid.rename_column('labels.real-label','label')
klue_sts_test = klue_sts_test.flatten()
klue_sts_test = klue_sts_test.rename_column('labels.real-label','label')

In [87]:
# 최대 길이 설정 
max_length = 512

# 모델 입력으로 만들어 주는 전처리 함수 
# truncation으로 길이 넘어가면 자를 수 있게 설정, padding 으로 길이 맞춰 주기 
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'],
                     truncation=True, max_length=max_length, padding=True)


In [88]:
# 전처리 함수 각 데이터셋에 적용 
encoded_train = klue_sts_train.map(preprocess_function, batched=True)
encoded_valid = klue_sts_valid.map(preprocess_function, batched=True)
encoded_test = klue_sts_test.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classif

In [28]:
# 텐서보드 dir 정의 
logdir_path = os.path.join(_CUR_DIR, "sts_log")# log 파일을 저장할 경로를 지정
writer = SummaryWriter(logdir_path) 

In [29]:
##### 베이스 라인 모델: batch_size= 32, 나머지는 default

output_dir = os.path.join(_CUR_DIR, "sts_output")

In [30]:
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy = "epoch", # 각 epoch 마지막에 저장 
    load_best_model_at_end=True, # train 종료시 best model 로드할지 여부

    # Dataset
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    

    # Evaluation 
    evaluation_strategy = "epoch",# 각 epoch 마지막에 평가

    # Randomness
    seed=seed,
)

In [31]:
# f1과 pearson 메트릭 불러오기 
metric_f1 = load_metric("f1") 
metric_pearson = load_metric("pearsonr") 

In [32]:
# 출력시 메트릭 정의 하는 함수 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    predict_f1 = [1 if i>=3.0 else 0 for i in predictions] # 3.0 이상-> 1, 3.0 미만 -> 0으로 변환
    labels_f1 = [1 if i>=3.0 else 0 for i in labels] # 3.0 이상-> 1, 3.0 미만 -> 0으로 변환

    pr = metric_pearson.compute(predictions=predictions, # peason r 계산
                                   references=labels)

    f1 = metric_f1.compute(predictions=predict_f1,   # f1
                                     references=labels_f1)
    return {'pearsonr' : pr,
             'f1' : f1}

In [89]:
# trainer 정의 
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience = patience)] # earlystopping 추가
)

In [90]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, guid, labels.binary-label, sentence1, source, labels.label. If sentence2, guid, labels.binary-label, sentence1, source, labels.label are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10501
  Num Epochs = 25
  Instantaneous batch size per device = 58
  Total train batch size (w. parallel, distributed & accumulation) = 58
  Gradient Accumulation steps = 1
  Total optimization steps = 4550


Epoch,Training Loss,Validation Loss,Pearsonr,F1
1,No log,0.168103,{'pearsonr': 0.9726021954081557},{'f1': 0.9541284403669724}
2,No log,0.145495,{'pearsonr': 0.97674812416218},{'f1': 0.9548387096774194}
3,0.305200,0.216145,{'pearsonr': 0.9753702289793063},{'f1': 0.9565217391304348}
4,0.305200,0.296853,{'pearsonr': 0.9719130964833813},{'f1': 0.9467248908296944}


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, guid, labels.binary-label, sentence1, source, labels.label. If sentence2, guid, labels.binary-label, sentence1, source, labels.label are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
Trainer is attempting to log a value of "{'pearsonr': 0.9726021954081557}" of type <class 'dict'> for key "eval/pearsonr" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9541284403669724}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /content/drive/MyDrive/AIBootCamp/NLP/sts_output/checkpoin

TrainOutput(global_step=728, training_loss=0.23985145904205657, metrics={'train_runtime': 865.6113, 'train_samples_per_second': 303.283, 'train_steps_per_second': 5.256, 'total_flos': 2719398951615432.0, 'train_loss': 0.23985145904205657, 'epoch': 4.0})

In [91]:
# baseline score
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, guid, labels.binary-label, sentence1, source, labels.label. If sentence2, guid, labels.binary-label, sentence1, source, labels.label are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64


Trainer is attempting to log a value of "{'pearsonr': 0.97674812416218}" of type <class 'dict'> for key "eval/pearsonr" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9548387096774194}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 4.0,
 'eval_f1': {'f1': 0.9548387096774194},
 'eval_loss': 0.14549507200717926,
 'eval_pearsonr': {'pearsonr': 0.97674812416218},
 'eval_runtime': 6.9067,
 'eval_samples_per_second': 168.966,
 'eval_steps_per_second': 2.751}

In [92]:
# test set으로 prediction 
predictions = trainer.predict(encoded_test)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, guid, labels.binary-label, sentence1, source, labels.label. If sentence2, guid, labels.binary-label, sentence1, source, labels.label are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 519
  Batch size = 64


In [93]:
# test set score 
predictions.metrics

{'test_f1': {'f1': 0.814498933901919},
 'test_loss': 0.4114389419555664,
 'test_pearsonr': {'pearsonr': 0.9010864050775359},
 'test_runtime': 3.0923,
 'test_samples_per_second': 167.834,
 'test_steps_per_second': 2.91}